# Capstone Project: 2025 NBA Playoffs Q&A

## Introduction

- This project demonstrates the use of advanced AI capabilities to answer questions about the **2025 NBA Playoffs**. 
- It leverages the Gemini API Python SDK and integrates tools like Google Search for grounding and citation generation. 
- The notebook showcases how to interact with AI models, evaluate their responses, and provide grounded, citation-backed answers to user queries.

---

## Capabilities Demonstrated

### 1. **Package Installation and Setup**
- Installs required packages (`google-genai`, `google-api-core`).
- Imports necessary modules and initializes the Gemini API client using an API key.

### 2. **Automated Retry Mechanism**
- Implements a retry policy to handle quota limits or temporary API errors during model calls.

### 3. **Model Invocation**
- Defines and uses the `gemini-2.0-flash` model for generating responses.
- Supports both few-shot prompting and grounding with external tools like Google Search.

### 4. **Few-Shot Prompting**
- Constructs a detailed prompt to simulate an NBA analyst answering questions about the 2025 NBA Playoffs.
- Handles cases where the model cannot find information by responding with "I don't know."

### 5. **Grounded Responses**
- Enhances responses by integrating search grounding using Google Search.
- Uses Context caching for 5 minutes for cost efficiency
- Provides citations with sources and links for the generated answers.

### 6. **Evaluation of AI Responses**
- Defines a structured evaluation prompt to assess the quality of AI-generated responses.
- Evaluates responses based on criteria like instruction following, groundedness, conciseness, and fluency.
- Assigns ratings using a predefined rubric and provides step-by-step explanations for the evaluation.

### 7. **Citation Generation**
- Processes data to generate citations for answers, ensuring transparency and reliability.

---

## Problem Being Solved

The notebook addresses the challenge of providing accurate, grounded, and citation-backed answers to questions about the 2025 NBA Playoffs. It demonstrates how to:
- Use AI models to generate informative responses.
- Enhance responses with external data sources for grounding.
- Evaluate and improve the quality of AI-generated content.
- Provide users with reliable and transparent answers, including citations for verification.

This workflow is particularly useful for scenarios requiring expert-level insights, grounded information, and high-quality content generation, such as sports analysis, journalism, and research.

---

## Gen AI Capabilities used

- Few-shot prompting
- Grounding
- Gen AI evaluation

## Notebook Run Steps

### 1. Install packages
Start by installing and importing the Gemini API Python SDK.

In [1]:
# Install the google-genai, google-adk and google-api-core
!uv pip install -qU google-genai google-adk google-api-core

### 2. import packages

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.11.0'

### 3. Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

If the `GOOGLE_API_KEY` is not available through Kaggle secrets, prompt to enter the key using `getpass` 

In [3]:
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
except Exception:
    import getpass
    GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

client = genai.Client(api_key=GOOGLE_API_KEY)

### 4. Setup Automated retry

In [4]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

### 5. Define `model_name` to be invoked

In [5]:
model_name = 'gemini-2.0-flash'

### 6. Define `few_show_prompt` and `question`


In [32]:
few_show_prompt = """
       Hello, You are an NBA analyst with deep expertise in NBA statistics.
         1. You are required to answer questions about the 2025 playoff teams.
         2. If you are not able to find the information, respond with "I don't know."
         3. Provide citations with sources and links in the answer.
       """ 

question = "Which teams are in the 2025 NBA playoffs?"

input_content = few_show_prompt + question


### 7. Test the `few_show_prompt` without using Grounding
Here we will append a `question` to the `few_show_prompt` 

In [33]:

# Ask for information without search grounding.
response = client.models.generate_content(
    model=model_name,
    contents=input_content)

few_shot_response_text = response.text
# Display the response
# Format the response as a Markdown cell
few_shot_formatted_response = f"### {few_shot_response_text}"
Markdown(few_shot_formatted_response)

### As an NBA analyst, I'd love to tell you which teams are in the 2025 NBA playoffs! However, **I don't know** that information yet. The 2025 playoffs haven't happened, and predicting the exact teams that will qualify so far in advance is impossible due to player development, injuries, trades, and other unforeseen circumstances.


### 8. Use Grounding grounding `google_search`

In [39]:
# And now re-run the same query with search grounding enabled.

config_with_google_search = types.GenerateContentConfig(tools=[types.Tool(google_search=types.GoogleSearch())])

def query_with_grounding():
    response = client.models.generate_content(
        model=model_name,
        contents=input_content,
        config=config_with_google_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
grounded_response_text = rc.content.parts[0].text
grounded_formatted_response = f"### {grounded_response_text}"
Markdown(grounded_formatted_response)

### The 2025 NBA playoffs are underway, and the bracket is officially set. Here are the teams that have made it to the first round:

**Eastern Conference**

*   (1) Cleveland Cavaliers vs. (8) Miami Heat
*   (2) Boston Celtics vs. (7) Orlando Magic
*   (3) New York Knicks vs. (6) Detroit Pistons
*   (4) Indiana Pacers vs. (5) Milwaukee Bucks

**Western Conference**

*   (1) Oklahoma City Thunder vs. (8) Memphis Grizzlies
*   (2) Houston Rockets vs. (7) Golden State Warriors
*   (3) Los Angeles Lakers vs. (6) Minnesota Timberwolves
*   (4) Denver Nuggets vs. (5) Los Angeles Clippers

Sources:

*   [https://www.cbssports.com/nba/news/2025-nba-playoff-bracket-first-round-matchups-schedule-game-times-as-heat-and-grizzlies-get-final-spots/](https://www.cbssports.com/nba/news/2025-nba-playoff-bracket-first-round-matchups-schedule-game-times-as-heat-and-grizzlies-get-final-spots/)
*   [https://www.youtube.com/watch?v=BwXzs5GzK9I](https://www.youtube.com/watch?v=BwXzs5GzK9I)
*   [https://en.wikipedia.org/wiki/2025_NBA_playoffs](https://en.wikipedia.org/wiki/2025_NBA_playoffs)


### 9. Gen AI Evaluation

Evaluate the quality of the response generated by the AI model


In [40]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. 
The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. 
The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, ai_response):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(
    model=model_name,
    config=config_with_google_search)

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=input_content, ai_response=grounded_response_text)


In [41]:
# Format the response by steps
def format_steps(steps):
    formatted_steps = []
    for i, step in enumerate(steps.split('\n')):
        if step.strip():
            formatted_steps.append(f"{i + 1}: {step.strip()}\n")
    return "\n".join(formatted_steps)
eval_formatted_steps = format_steps(text_eval)
eval_formatted_response = f"### Evaluation Steps\n\n{eval_formatted_steps}"
Markdown(eval_formatted_response)


### Evaluation Steps

1: STEP 1: The response provided an accurate list of teams in the 2025 NBA playoffs. The links provided are relevant to the answer.

2: STEP 2: The response follows instructions, is grounded, is concise, and fluent.

4: Rating: 5
